# Insert here your bidding quantity list :

In [1]:
import pandas as pd
from pandas import ExcelFile

# In my example the bidding quantity are taken from the Wind Data :
BidData = pd.read_excel('WindForecast_DA_12.xlsx')
print(BidData.columns)                                         # To check the import

BID = BidData['fore']

Index(['day', 'hour', 'dato', 'dati', 'ForeTime', 'hors', 'meas', 'fore', 'q5',
       'q10', 'q15', 'q20', 'q25', 'q30', 'q35', 'q40', 'q45', 'q50', 'q55',
       'q60', 'q65', 'q70', 'q75', 'q80', 'q85', 'q90', 'q95'],
      dtype='object')


# Calculation code :

### Data :

#### Import Data :

In [2]:
# Import data for measurements and forecast made at 12:00 Day-1 :
WindData = pd.read_excel('WindForecast_DA_12.xlsx')

# Import market price data for 2017 from Nordpool :
SpotData = pd.read_excel('DK1_2017.xlsx')

# Check : Print the name of the excel file columns
print(WindData.columns)
print(SpotData.columns)

Index(['day', 'hour', 'dato', 'dati', 'ForeTime', 'hors', 'meas', 'fore', 'q5',
       'q10', 'q15', 'q20', 'q25', 'q30', 'q35', 'q40', 'q45', 'q50', 'q55',
       'q60', 'q65', 'q70', 'q75', 'q80', 'q85', 'q90', 'q95'],
      dtype='object')
Index(['Date', 'Hours', 'Clearing', 'Up', 'Down', 'Unnamed: 5', 'Unnamed: 6'], dtype='object')


#### Extract meaningful parts :

In [3]:
# Wind power measure, in kW :
Measure = WindData['meas']

# Market clearing price, in €/MWh :
LambdaS = SpotData['Clearing']

# Extract market regulation price, in €/MWh :
LambdaUP = SpotData['Up']
LambdaDOWN = SpotData['Down']

# Number of time units :
L = len(Measure)

#### Market properties :

We build an indicator dP which will give the sign for the type of regulation needed for the market:

#dP = -1 for down-regulation

#dP = 1 for up-regulation

#dP = 0 when the market is perfectly balanced

In [4]:
dP = [0]*L
for k in range (0, L, 1) :
    lambda_up = LambdaUP[k]
    lambda_down = LambdaDOWN[k]
    lambdaS = LambdaS[k]
    
    up = lambda_up - lambdaS      # lambda_up >= lambdaS
    down = lambdaS - lambda_down  # lambda_down <= lambdaS
    
    if lambda_up == lambda_down:
        dP[k] = 0
    elif up > down :
        dP[k] = -1                # Need of upward regulation, negative imbalance
    else :
        dP[k] = 1                 # Need of downward regulation, positiv imbalance


Now we can determine the balancing price LambdaB according to the market imbalance :

In [5]:
LambdaB = [0]*L
for k in range (0, L, 1):
    dp = dP[k]
    if dp < 0 :
        LambdaB[k] = LambdaUP[k]     # Upward regulation
    elif dp > 0 :
        LambdaB[k] = LambdaDOWN[k]   # Downward regulation
    else :
        LambdaB[k] = LambdaS[k]      # Balanced Market


Adapt bid and real values when the producer is not scheduled :

In [6]:
RealBID = [0]*L
RealMEAS = [0]*L

for k in range (0,L,1):
    if LambdaS[k] < 0:
        RealBID[k] = 0
        RealMEAS[k] = 0
    else :
        RealBID[k] = BID[k]
        RealMEAS[k]= Measure[k]

#### Individual imbalance :

An indicator of the producer imbalance is defined similarly to the Market imbalance indicator dP :

In [7]:
SURPLUS = [0]*L      # Production surplus at time t, in kW
LACK = [0]*L         # Lack of production at time t, in kW
IMBAL = [0]*L        # Imbalance indicator

for k in range (0, L, 1) :
    real = RealMEAS[k]
    bid = RealBID[k]
    
    if real > bid :
        IMBAL[k] = 1              # The producer produces more than forecasted, his imbalance is positiv.
        SURPLUS[k] = real-bid
        LACK[k] = 0
    elif real < bid :
        IMBAL[k] = -1             # The producer doesn't produce enough, his imbalance is negativ.
        SURPLUS[k] = 0
        LACK[k] = real-bid       
    else :
        IMBAL[k] = 0              # The production bid was exact.
        SURPLUS[k] = 0
        LACK[k] = 0

#### Day Ahead Revenue :

In [8]:
RevDA = [0]*L

for k in range (0, L, 1):
    
    if LambdaS[k] >= 0 :
        lambdaS = LambdaS[k]          # 0.001 factor due to conversion from MWh to kWh
        bid = RealBID[k]
        RevDA[k] = lambdaS*bid*0.001

#### Balancing Market Revenue :

In [9]:
RevB = [0]*L              # Revenue of balancing market
LambdaREG = [0]*L         # Regulation price applied, LambdaS or LambdaB

for k in range (0, L, 1):
    real = RealMEAS[k]
    bid = RealBID[k]
    market_imbal = dP[k]
    indiv_imbal = IMBAL[k]
    
    lambdaB = LambdaB[k]
    lambdaS = LambdaS[k]
    
    # Case of upward regulation, the producer helps if he produces more than forecasted ie. his imbalance is positiv
    if market_imbal < 0 :
        if indiv_imbal >= 0 :
            revB = lambdaS*(real-bid)*0.001
            lambdaREG = lambdaS
        else :
            revB = lambdaB*(real-bid)*0.001
            lambdaREG = lambdaB
    # Case of downward regualtion, the producer helps if he produces less than forecasted ie. his imbalance is negativ
    elif market_imbal > 0 :
        if indiv_imbal <= 0 :
            revB = lambdaS*(real-bid)*0.001
            lambdaREG = lambdaS
        else :
            revB = lambdaB*(real-bid)*0.001
            lambdaREG = lambdaB
    # Case of balanced market, the producer's imbalance is not penalized :
    else :
        revB = lambdaS*(real-bid)*0.001
        lambdaREG = lambdaS
    
    RevB[k] = revB
    LambdaREG[k] = lambdaREG

# Results

In [10]:
# Net revenue :
RevNET = [0]*L
for k in range (0,L, 1):
    RevNET[k] = RevDA[k] + RevB[k]

RevNET_annual = sum(RevNET)
print('The net annual revenue is, in €:')
print(RevNET_annual)
print('Composed of a day-ahead revenue, in €:')
print(sum(RevDA))
print('and a balancing revenue, in €:')
print(sum(RevB))

The net annual revenue is, in €:
19151045.13385997
Composed of a day-ahead revenue, in €:
20564379.207590017
and a balancing revenue, in €:
-1413334.073729998


In [11]:
# Real generation VS scheduled generation :
RealGene = sum(RealMEAS)*0.001
SchedGene = sum(RealBID)*0.001
print('The annual real generation is, in MWh:')
print(RealGene)
print('Against a annual scheduled generation of, in MWh:')
print(SchedGene)

The annual real generation is, in MWh:
708448.295
Against a annual scheduled generation of, in MWh:
731385.884


In [12]:
# Surplus VS Shortage :
TotSurplus = sum(SURPLUS)*0.001
TotShortage = sum(LACK)*0.001
print('The global generation surplus in 2017 is, in MWh:')
print(TotSurplus)
print('& the global generation shortage in 2017 is, in MWh:')
print(TotShortage)
print('Representing a share of the global production, in %:')
print(100*(TotSurplus-TotShortage)/RealGene)

The global generation surplus in 2017 is, in MWh:
72142.35800000001
& the global generation shortage in 2017 is, in MWh:
-95079.947
Representing a share of the global production, in %:
23.604023918216924


In [13]:
# Imbalance cost :

# Cost of upward regulation, when the producer is in shortage :
UPCOST = [0]*L
# Revenue when downward regulation, when the producer is in surplus :
DOWNCOST = [0]*L

# Opportunity loss cost : when the producer generates less than schedule and have to buy electricity more expensive than its own : :
OPCOST_SHORTAGE = [0]*L
# Opportunity loss cost : when the producer generates more than schedule and sell it for <= price than lambdaS :
OPCOST_SURPLUS = [0]*L

for k in range (0,L,1):
    UPCOST[k]=0.001*LACK[k]*LambdaREG[k]
    DOWNCOST[k]=0.001*SURPLUS[k]*LambdaREG[k]
    
    OPCOST_SHORTAGE[k]=0.001*LACK[k]*(LambdaREG[k]-LambdaS[k])
    OPCOST_SURPLUS[k]=0.001*SURPLUS[k]*(LambdaREG[k]-LambdaS[k])
    
print('The global shortage over the year create a opportunity loss of, in €:')
print(sum(OPCOST_SHORTAGE))
print('The global surplus over the year create a opportunity loss of, in €:')
print(sum(OPCOST_SURPLUS))
print('Giving a global opportunity loss of, in € :')
print(sum(OPCOST_SHORTAGE)+sum(OPCOST_SURPLUS))

# Just to check : The global opportunity loss should be equal to 
# the difference between the net revenue of this strategy and the ideal revenue.

The global shortage over the year create a opportunity loss of, in €:
-275288.26349000033
The global surplus over the year create a opportunity loss of, in €:
-289077.97755999974
Giving a global opportunity loss of, in € :
-564366.2410500001


In [14]:
# Average regulation prices per unit :
  
    # Upward and downward regulation, in €/MWh :
avUP = sum(UPCOST)/(0.001*sum(LACK))
avDOWN = sum(DOWNCOST)/(0.001*sum(SURPLUS))


print('The average upward regulation price is, in €/MWh:')
print(avUP)
print('The average downward regulation price is, in €/MWh:')
print(avDOWN)

print('The average energy price, in €/MWh:')
print(sum(RevNET)/RealGene)

The average upward regulation price is, in €/MWh:
34.77980090744041
The average downward regulation price is, in €/MWh:
26.247098178021883
The average energy price, in €/MWh:
27.032382276902748


In [15]:
# Performance ratio
Perf = sum(RevNET)/19715411.37491005
print('The performance ratio of this strategy is:')
print(Perf)

The performance ratio of this strategy is:
0.9713743613907901


# Results analysis

#### Import useful packages

In [16]:
from matplotlib import pyplot as plt
import numpy as np
import math

## Cumulative Net Revenue

In [23]:
CumulRevNET = [0]*L
CumulRevNET[0] = RevNET[0]
for k in range (1,L,1):
    CumulRevNET[k] = CumulRevNET[k-1] + RevNET[k]
    
# Export all revenues data to Excel :
    
df2 = pd.DataFrame([RevDA, RevB, RevNET, CumulRevNET])
df2.to_excel("Revenue2.xlsx")
print('Exported')


Exported


## Forecast reliability

In [1]:
ERROR = [0]*L
DeltaLAMBDA = [0]*L                                # Difference between clearing and applied balancing price under 2-price settlement
relERROR = [0]*L                                   # Relative error with the measure
for k in range (0,L,1):
    ERROR[k] = (SURPLUS[k] + LACK[k])*0.001
    DeltaLAMBDA[k] = LambdaREG[k] - LambdaS[k]
    realMeas = RealMEAS[k]*0.001
    if realMeas == 0:
        relERROR[k] = 0
    else:
        relERROR [k] = ERROR[k]/realMeas

relError = (sum(relERROR)/L)
print('The medium relative forecast error is :')
print(relError)


NameError: name 'L' is not defined